# ESDP1 – Homework 1
## ERA5 Data Access (Pelin Su Kaplan)

In [ ]:
import cdsapi
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

# Step 1: CDS API configuration

# Step 2: ERA5 download request

# Step 3: Open dataset with xarray

# Step 4: Inspect dataset

# Step 5: Basic plots

# Step 6: Summary